In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from collections import Counter
from imblearn.over_sampling import SMOTE
sm= SMOTE(random_state= 15)

X_sm, y_sm= sm.fit_resample(X, y)
print(f'SMOTE: {Counter(y_sm)}')

SMOTE: Counter({0: 42582, 1: 42582})


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

X_sm_train, y_sm_train= sm.fit_resample(X_train, y_train)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
lr_sm= LogisticRegression(random_state=15)
lr_sm.fit(X_sm_train, y_sm_train)
y_pred_lr_sm= lr_sm.predict(X_test)
y_pred_prob_lr_sm= lr_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_sm, labels= lr_sm.classes_)
print(classification_report(y_test, y_pred_lr_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr_sm))

              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.26      0.81      0.39       912

    accuracy                           0.75      9429
   macro avg       0.61      0.78      0.62      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.25565217391304346
Accuracy: 0.7542687453600594
AUC: 0.8519174241815647


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_sm= DecisionTreeClassifier(random_state=15)
dt_sm.fit(X_sm_train, y_sm_train)
y_pred_dt_sm= dt_sm.predict(X_test)
y_pred_prob_dt_sm= dt_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_sm, labels= dt_sm.classes_)
print(classification_report(y_test, y_pred_dt_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_sm))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      8517
           1       0.24      0.28      0.26       912

    accuracy                           0.85      9429
   macro avg       0.58      0.59      0.59      9429
weighted avg       0.86      0.85      0.85      9429

Precision: 0.24357754519505234
Accuracy: 0.8461130554671757
AUC: 0.5928349698950911


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_sm= GridSearchCV(estimator= dt_sm, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_sm.fit(X_sm_train, y_sm_train)

print("Best Parameters:", gs_dt_sm.best_params_)
print("Best Precision Score:", gs_dt_sm.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Precision Score: 0.9419633844621199


In [12]:
tuned_dt_sm= gs_dt_sm.best_estimator_
y_pred_tuned_dt_sm= tuned_dt_sm.predict(X_test)
y_pred_prob_tuned_dt_sm= tuned_dt_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_sm, labels= tuned_dt_sm.classes_)
print(classification_report(y_test, y_pred_tuned_dt_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_sm))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      8517
           1       0.29      0.22      0.25       912

    accuracy                           0.87      9429
   macro avg       0.61      0.58      0.59      9429
weighted avg       0.86      0.87      0.87      9429

Precision: 0.29411764705882354
Accuracy: 0.8735815038710362
AUC: 0.7224116653174559


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_sm= KNeighborsClassifier()
knn_sm.fit(X_sm_train, y_sm_train)
y_pred_knn_sm= knn_sm.predict(X_test)
y_pred_prob_knn_sm= knn_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_sm, labels= knn_sm.classes_)
print(classification_report(y_test, y_pred_knn_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_sm))

              precision    recall  f1-score   support

           0       0.95      0.78      0.85      8517
           1       0.22      0.58      0.32       912

    accuracy                           0.76      9429
   macro avg       0.58      0.68      0.59      9429
weighted avg       0.88      0.76      0.80      9429

Precision: 0.21842863019333608
Accuracy: 0.7580867536324106
AUC: 0.7390220848293094


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_sm= GridSearchCV(estimator= knn_sm, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_sm.fit(X_sm_train, y_sm_train)
print("Best Parameters:", gs_knn_sm.best_params_)
print("Best Precision Score:", gs_knn_sm.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.9306005964235627


In [15]:
tuned_knn_sm= gs_knn_sm.best_estimator_
y_pred_tuned_knn_sm= tuned_knn_sm.predict(X_test)
y_pred_prob_tuned_knn_sm= tuned_knn_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_sm, labels= tuned_knn_sm.classes_)
print(classification_report(y_test, y_pred_tuned_knn_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_sm))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      8517
           1       0.25      0.19      0.22       912

    accuracy                           0.87      9429
   macro avg       0.58      0.57      0.57      9429
weighted avg       0.85      0.87      0.86      9429

Precision: 0.24964739069111425
Accuracy: 0.8656273199703044
AUC: 0.6226777610928813


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_sm= GaussianNB()
nb_sm.fit(X_sm_train, y_sm_train)
y_pred_nb_sm= nb_sm.predict(X_test)
y_pred_prob_nb_sm= nb_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_sm, labels= nb_sm.classes_)
print(classification_report(y_test, y_pred_nb_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_sm))

              precision    recall  f1-score   support

           0       0.97      0.55      0.70      8517
           1       0.17      0.86      0.28       912

    accuracy                           0.58      9429
   macro avg       0.57      0.70      0.49      9429
weighted avg       0.90      0.58      0.66      9429

Precision: 0.16881998277347116
Accuracy: 0.5770495280517552
AUC: 0.7982750314644189


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_sm= SVC(kernel= 'rbf', gamma= 1, probability= True, random_state=15)
svc_sm.fit(X_sm_train, y_sm_train)
y_pred_svc_sm= svc_sm.predict(X_test)
y_pred_prob_svc_sm= svc_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_sm, labels= svc_sm.classes_)
print(classification_report(y_test, y_pred_svc_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_sm))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      8517
           1       0.15      0.04      0.06       912

    accuracy                           0.89      9429
   macro avg       0.53      0.51      0.50      9429
weighted avg       0.83      0.89      0.86      9429

Precision: 0.15458937198067632
Accuracy: 0.8881111464630396
AUC: 0.6973222028595029


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_sm= RandomForestClassifier(random_state=15)
rf_sm.fit(X_sm_train, y_sm_train)
y_pred_rf_sm= rf_sm.predict(X_test)
y_pred_prob_rf_sm= rf_sm.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_sm, labels= rf_sm.classes_)
print(classification_report(y_test, y_pred_rf_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_sm))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8517
           1       0.39      0.14      0.21       912

    accuracy                           0.90      9429
   macro avg       0.65      0.56      0.58      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.3944954128440367
Accuracy: 0.8959592745784283
AUC: 0.8239861221828788


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_sm= GridSearchCV(estimator= rf_sm, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_sm.fit(X_sm_train, y_sm_train)
print("Best Parameters:", gs_tuned_rf_sm.best_params_)
print("Best Precision Score:", gs_tuned_rf_sm.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Best Precision Score: 0.974182525022435


In [21]:
tuned_rf_sm= gs_tuned_rf_sm.best_estimator_
y_pred_tuned_rf_sm= tuned_rf_sm.predict(X_test)
y_pred_prob_tuned_rf_sm= tuned_rf_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_sm, labels= tuned_rf_sm.classes_)
print(classification_report(y_test, y_pred_tuned_rf_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_sm))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.41      0.13      0.20       912

    accuracy                           0.90      9429
   macro avg       0.66      0.56      0.57      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.410958904109589
Accuracy: 0.8977622229292608
AUC: 0.8343861168272331


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_sm= AdaBoostClassifier(random_state=15)
ada_sm.fit(X_sm_train, y_sm_train)
y_pred_ada_sm= ada_sm.predict(X_test)
y_pred_prob_ada_sm= ada_sm.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_sm, labels= ada_sm.classes_)
print(classification_report(y_test, y_pred_ada_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_sm))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      8517
           1       0.37      0.45      0.41       912

    accuracy                           0.87      9429
   macro avg       0.66      0.69      0.67      9429
weighted avg       0.89      0.87      0.88      9429

Precision: 0.37398373983739835
Accuracy: 0.8736875596563792
AUC: 0.8426422438919889


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_sm= GridSearchCV(estimator= ada_sm, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_sm.fit(X_sm_train, y_sm_train)
print("Best Parameters:", gs_ada_sm.best_params_)
print("Best Precision Score:", gs_ada_sm.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9610610256368908


In [25]:
tuned_ada_sm= gs_ada_sm.best_estimator_
y_pred_tuned_ada_sm= tuned_ada_sm.predict(X_test)
y_pred_prob_tuned_ada_sm= tuned_ada_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_sm, labels= tuned_ada_sm.classes_)
print(classification_report(y_test, y_pred_tuned_ada_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_sm))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      8517
           1       0.47      0.27      0.34       912

    accuracy                           0.90      9429
   macro avg       0.70      0.62      0.64      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.4669187145557656
Accuracy: 0.8995651712800933
AUC: 0.8485557265242478


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_sm= GradientBoostingClassifier(random_state=15)
grb_sm.fit(X_sm_train, y_sm_train)
y_pred_grb_sm= grb_sm.predict(X_test)
y_pred_prob_grb_sm= grb_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_grb_sm, labels= grb_sm.classes_)
print(classification_report(y_test, y_pred_grb_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_sm))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      8517
           1       0.41      0.33      0.37       912

    accuracy                           0.89      9429
   macro avg       0.67      0.64      0.65      9429
weighted avg       0.88      0.89      0.88      9429

Precision: 0.4139344262295082
Accuracy: 0.8899140948138721
AUC: 0.8439502573799769


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_sm= GridSearchCV(estimator= grb_sm, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_sm.fit(X_sm_train, y_sm_train)

print("Best Parameters:", gs_grb_sm.best_params_)
print("Best Precision Score:", gs_grb_sm.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.9710000002981778


In [28]:
tuned_grb_sm= gs_grb_sm.best_estimator_
y_pred_tuned_grb_sm= tuned_grb_sm.predict(X_test)
y_pred_prob_tuned_grb_sm= tuned_grb_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_sm, labels= tuned_grb_sm.classes_)
print(classification_report(y_test, y_pred_tuned_grb_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_sm))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.48      0.21      0.29       912

    accuracy                           0.90      9429
   macro avg       0.70      0.59      0.62      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.4846153846153846
Accuracy: 0.9020044543429844
AUC: 0.8460542150992134


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_sm= XGBClassifier(random_state=15)
xgb_sm.fit(X_sm_train, y_sm_train)
y_pred_xgb_sm= xgb_sm.predict(X_test)
y_pred_prob_xgb_sm= xgb_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_sm, labels= xgb_sm.classes_)
print(classification_report(y_test, y_pred_xgb_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_sm))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.45      0.14      0.21       912

    accuracy                           0.90      9429
   macro avg       0.68      0.56      0.58      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.4548736462093863
Accuracy: 0.9006257291335242
AUC: 0.8326054289769275


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_sm= GridSearchCV(estimator= xgb_sm, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_sm.fit(X_sm_train, y_sm_train)

print("Best Parameters:", gs_xgb_sm.best_params_)
print("Best Precision Score:", gs_xgb_sm.best_score_)

tuned_xgb_sm= gs_xgb_sm.best_estimator_
y_pred_tuned_xgb_sm= tuned_xgb_sm.predict(X_test)
y_pred_prob_tuned_xgb_sm= tuned_xgb_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_sm, labels= tuned_xgb_sm.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_sm))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9687826877140703
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      8517
           1       0.48      0.22      0.30       912

    accuracy                           0.90      9429
   macro avg       0.70      0.60      0.62      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.476303317535545
Accuracy: 0.9011560080602397
AUC: 0.844232523085923


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_sm= LGBMClassifier(random_state=15)
lgm_sm.fit(X_sm_train, y_sm_train)
y_pred_lgm_sm= lgm_sm.predict(X_test)
y_pred_prob_lgm_sm= lgm_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_sm, labels= lgm_sm.classes_)
print(classification_report(y_test, y_pred_lgm_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_sm))

[LightGBM] [Info] Number of positive: 34065, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017791 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the train set: 68130, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.51      0.15      0.23       912

    accuracy                           0.90      9429
   macro avg       0.71      0.57      0.59      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5075187969924813
Accuracy: 0.9037013469084738
AUC: 0.8442567908558527


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_sm= GridSearchCV(estimator= lgm_sm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_sm.fit(X_sm_train, y_sm_train)

print("Best Parameters:", gs_lgm_sm.best_params_)
print("Best Precision Score:", gs_lgm_sm.best_score_)

tuned_lgm_sm= gs_lgm_sm.best_estimator_
y_pred_tuned_lgm_sm= tuned_lgm_sm.predict(X_test)
y_pred_prob_tuned_lgm_sm= tuned_lgm_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_sm, labels= tuned_lgm_sm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_sm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Number of positive: 34065, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the train set: 68130, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 100, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.9799104239143718
[LightGBM] [Warning] min_data_in_leaf is set=

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_sm= CatBoostClassifier(random_state=15)
cat_sm.fit(X_sm_train, y_sm_train)
y_pred_cat_sm= cat_sm.predict(X_test)
y_pred_prob_cat_sm= cat_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_sm, labels= cat_sm.classes_)
print(classification_report(y_test, y_pred_cat_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_sm))

Learning rate set to 0.062484
0:	learn: 0.6249055	total: 183ms	remaining: 3m 2s
1:	learn: 0.5691880	total: 225ms	remaining: 1m 52s
2:	learn: 0.5270046	total: 261ms	remaining: 1m 26s
3:	learn: 0.4918651	total: 295ms	remaining: 1m 13s
4:	learn: 0.4609432	total: 330ms	remaining: 1m 5s
5:	learn: 0.4293887	total: 367ms	remaining: 1m
6:	learn: 0.4061425	total: 407ms	remaining: 57.7s
7:	learn: 0.3877623	total: 441ms	remaining: 54.7s
8:	learn: 0.3745059	total: 477ms	remaining: 52.5s
9:	learn: 0.3588853	total: 513ms	remaining: 50.8s
10:	learn: 0.3480686	total: 550ms	remaining: 49.4s
11:	learn: 0.3360262	total: 586ms	remaining: 48.3s
12:	learn: 0.3285224	total: 623ms	remaining: 47.3s
13:	learn: 0.3150728	total: 659ms	remaining: 46.4s
14:	learn: 0.3088232	total: 694ms	remaining: 45.6s
15:	learn: 0.3004437	total: 730ms	remaining: 44.9s
16:	learn: 0.2927733	total: 765ms	remaining: 44.2s
17:	learn: 0.2868389	total: 798ms	remaining: 43.5s
18:	learn: 0.2774621	total: 834ms	remaining: 43.1s
19:	learn: 

164:	learn: 0.1546179	total: 7.39s	remaining: 37.4s
165:	learn: 0.1544983	total: 7.43s	remaining: 37.3s
166:	learn: 0.1543961	total: 7.47s	remaining: 37.3s
167:	learn: 0.1543095	total: 7.5s	remaining: 37.2s
168:	learn: 0.1542290	total: 7.54s	remaining: 37.1s
169:	learn: 0.1541139	total: 7.58s	remaining: 37s
170:	learn: 0.1539760	total: 7.61s	remaining: 36.9s
171:	learn: 0.1538963	total: 7.64s	remaining: 36.8s
172:	learn: 0.1537675	total: 7.68s	remaining: 36.7s
173:	learn: 0.1536892	total: 7.72s	remaining: 36.6s
174:	learn: 0.1534471	total: 7.75s	remaining: 36.6s
175:	learn: 0.1533800	total: 7.79s	remaining: 36.5s
176:	learn: 0.1532937	total: 7.82s	remaining: 36.4s
177:	learn: 0.1532260	total: 7.85s	remaining: 36.3s
178:	learn: 0.1526239	total: 7.89s	remaining: 36.2s
179:	learn: 0.1525514	total: 7.92s	remaining: 36.1s
180:	learn: 0.1524626	total: 7.96s	remaining: 36s
181:	learn: 0.1523363	total: 7.99s	remaining: 35.9s
182:	learn: 0.1521868	total: 8.03s	remaining: 35.8s
183:	learn: 0.152

325:	learn: 0.1363331	total: 13.3s	remaining: 27.5s
326:	learn: 0.1362665	total: 13.3s	remaining: 27.4s
327:	learn: 0.1361993	total: 13.4s	remaining: 27.4s
328:	learn: 0.1361483	total: 13.4s	remaining: 27.4s
329:	learn: 0.1360972	total: 13.4s	remaining: 27.3s
330:	learn: 0.1360691	total: 13.5s	remaining: 27.3s
331:	learn: 0.1360074	total: 13.5s	remaining: 27.2s
332:	learn: 0.1359788	total: 13.6s	remaining: 27.2s
333:	learn: 0.1359281	total: 13.6s	remaining: 27.1s
334:	learn: 0.1358667	total: 13.6s	remaining: 27.1s
335:	learn: 0.1357991	total: 13.7s	remaining: 27s
336:	learn: 0.1357241	total: 13.7s	remaining: 27s
337:	learn: 0.1356806	total: 13.7s	remaining: 26.9s
338:	learn: 0.1356364	total: 13.8s	remaining: 26.9s
339:	learn: 0.1355918	total: 13.8s	remaining: 26.8s
340:	learn: 0.1355190	total: 13.9s	remaining: 26.8s
341:	learn: 0.1353779	total: 13.9s	remaining: 26.7s
342:	learn: 0.1353346	total: 13.9s	remaining: 26.7s
343:	learn: 0.1352758	total: 14s	remaining: 26.6s
344:	learn: 0.1352

486:	learn: 0.1275787	total: 19.6s	remaining: 20.6s
487:	learn: 0.1275315	total: 19.6s	remaining: 20.6s
488:	learn: 0.1275022	total: 19.6s	remaining: 20.5s
489:	learn: 0.1274678	total: 19.7s	remaining: 20.5s
490:	learn: 0.1273743	total: 19.7s	remaining: 20.4s
491:	learn: 0.1273371	total: 19.7s	remaining: 20.4s
492:	learn: 0.1273034	total: 19.8s	remaining: 20.3s
493:	learn: 0.1272586	total: 19.8s	remaining: 20.3s
494:	learn: 0.1272160	total: 19.9s	remaining: 20.3s
495:	learn: 0.1271718	total: 19.9s	remaining: 20.2s
496:	learn: 0.1271262	total: 19.9s	remaining: 20.2s
497:	learn: 0.1270885	total: 20s	remaining: 20.1s
498:	learn: 0.1270361	total: 20s	remaining: 20.1s
499:	learn: 0.1270030	total: 20s	remaining: 20s
500:	learn: 0.1269607	total: 20.1s	remaining: 20s
501:	learn: 0.1268823	total: 20.1s	remaining: 20s
502:	learn: 0.1267895	total: 20.2s	remaining: 19.9s
503:	learn: 0.1267374	total: 20.2s	remaining: 19.9s
504:	learn: 0.1266857	total: 20.2s	remaining: 19.8s
505:	learn: 0.1266315	to

649:	learn: 0.1201732	total: 25.4s	remaining: 13.7s
650:	learn: 0.1201334	total: 25.5s	remaining: 13.7s
651:	learn: 0.1201010	total: 25.5s	remaining: 13.6s
652:	learn: 0.1200708	total: 25.6s	remaining: 13.6s
653:	learn: 0.1200366	total: 25.6s	remaining: 13.6s
654:	learn: 0.1200009	total: 25.6s	remaining: 13.5s
655:	learn: 0.1199511	total: 25.7s	remaining: 13.5s
656:	learn: 0.1198979	total: 25.7s	remaining: 13.4s
657:	learn: 0.1198290	total: 25.7s	remaining: 13.4s
658:	learn: 0.1197627	total: 25.8s	remaining: 13.3s
659:	learn: 0.1197295	total: 25.8s	remaining: 13.3s
660:	learn: 0.1197093	total: 25.8s	remaining: 13.3s
661:	learn: 0.1196734	total: 25.9s	remaining: 13.2s
662:	learn: 0.1196292	total: 25.9s	remaining: 13.2s
663:	learn: 0.1195871	total: 25.9s	remaining: 13.1s
664:	learn: 0.1195506	total: 26s	remaining: 13.1s
665:	learn: 0.1195222	total: 26s	remaining: 13s
666:	learn: 0.1194928	total: 26s	remaining: 13s
667:	learn: 0.1194580	total: 26.1s	remaining: 13s
668:	learn: 0.1194178	to

810:	learn: 0.1141900	total: 31s	remaining: 7.23s
811:	learn: 0.1141717	total: 31.1s	remaining: 7.19s
812:	learn: 0.1141490	total: 31.1s	remaining: 7.15s
813:	learn: 0.1141187	total: 31.1s	remaining: 7.11s
814:	learn: 0.1140691	total: 31.2s	remaining: 7.07s
815:	learn: 0.1140509	total: 31.2s	remaining: 7.04s
816:	learn: 0.1140268	total: 31.2s	remaining: 7s
817:	learn: 0.1140036	total: 31.3s	remaining: 6.96s
818:	learn: 0.1139659	total: 31.3s	remaining: 6.92s
819:	learn: 0.1139462	total: 31.3s	remaining: 6.88s
820:	learn: 0.1139058	total: 31.4s	remaining: 6.84s
821:	learn: 0.1138682	total: 31.4s	remaining: 6.8s
822:	learn: 0.1138330	total: 31.4s	remaining: 6.76s
823:	learn: 0.1138043	total: 31.5s	remaining: 6.72s
824:	learn: 0.1137667	total: 31.5s	remaining: 6.68s
825:	learn: 0.1137323	total: 31.6s	remaining: 6.65s
826:	learn: 0.1136995	total: 31.6s	remaining: 6.61s
827:	learn: 0.1136648	total: 31.6s	remaining: 6.57s
828:	learn: 0.1136332	total: 31.7s	remaining: 6.53s
829:	learn: 0.1136

970:	learn: 0.1091221	total: 37.7s	remaining: 1.13s
971:	learn: 0.1090981	total: 37.8s	remaining: 1.09s
972:	learn: 0.1090692	total: 37.8s	remaining: 1.05s
973:	learn: 0.1090504	total: 37.9s	remaining: 1.01s
974:	learn: 0.1090167	total: 37.9s	remaining: 972ms
975:	learn: 0.1089856	total: 38s	remaining: 934ms
976:	learn: 0.1089497	total: 38s	remaining: 896ms
977:	learn: 0.1089294	total: 38.1s	remaining: 857ms
978:	learn: 0.1088945	total: 38.2s	remaining: 819ms
979:	learn: 0.1088606	total: 38.2s	remaining: 780ms
980:	learn: 0.1088334	total: 38.3s	remaining: 742ms
981:	learn: 0.1088088	total: 38.4s	remaining: 703ms
982:	learn: 0.1087782	total: 38.4s	remaining: 664ms
983:	learn: 0.1087429	total: 38.5s	remaining: 626ms
984:	learn: 0.1087155	total: 38.5s	remaining: 587ms
985:	learn: 0.1086939	total: 38.6s	remaining: 548ms
986:	learn: 0.1086477	total: 38.6s	remaining: 509ms
987:	learn: 0.1086276	total: 38.7s	remaining: 470ms
988:	learn: 0.1086042	total: 38.8s	remaining: 431ms
989:	learn: 0.10

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_sm= GridSearchCV(estimator= cat_sm, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_sm.fit(X_sm_train, y_sm_train)

print("Best Parameters:", gs_cat_sm.best_params_)
print("Best Precision Score:", gs_cat_sm.best_score_)

tuned_cat_sm= gs_cat_sm.best_estimator_
y_pred_tuned_cat_sm= tuned_cat_sm.predict(X_test)
y_pred_prob_tuned_cat_sm= tuned_cat_sm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_sm, labels= tuned_cat_sm.classes_)
print(classification_report(y_test, y_pred_tuned_cat_sm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_sm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_sm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_sm))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.4861434	total: 28.3ms	remaining: 2.8s
1:	learn: 0.3893964	total: 55.1ms	remaining: 2.7s
2:	learn: 0.3295034	total: 83ms	remaining: 2.68s
3:	learn: 0.2883155	total: 111ms	remaining: 2.67s
4:	learn: 0.2720011	total: 135ms	remaining: 2.56s
5:	learn: 0.2472986	total: 163ms	remaining: 2.55s
6:	learn: 0.2376771	total: 185ms	remaining: 2.45s
7:	learn: 0.2300661	total: 212ms	remaining: 2.43s
8:	learn: 0.2164688	total: 240ms	remaining: 2.43s
9:	learn: 0.2022189	total: 267ms	remaining: 2.4s
10:	learn: 0.1939671	total: 295ms	remaining: 2.38s
11:	learn: 0.1901706	total: 318ms	remaining: 2.33s
12:	learn: 0.1818816	total: 345ms	remaining: 2.31s
13:	learn: 0.1798571	total: 371ms	remaining: 2.28s
14:	learn: 0.1781715	total: 394ms	remaining: 2.23s
15:	learn: 0.1719010	total: 421ms	remaining: 2.21s
16:	learn: 0.1653073	total: 447ms	remaining: 2.18s
17:	learn: 0.1645698	total: 471ms	remaining: 2.14s
18:	learn: 0.1641585	total: 494m

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_sm, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_sm, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_sm, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_sm, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_sm, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_sm, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_sm, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_sm, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_sm, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_sm, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_sm, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_sm, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_sm, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_sm, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_sm, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_sm, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_sm, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_sm, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_sm, zero_division= 0))

Logistic Regression Precision: 0.25565217391304346
Decision Tree Precision: 0.24357754519505234
Tuned Decision Tree Precision: 0.29411764705882354
KNeighborsClassifier Precision: 0.21842863019333608
Tuned KNeighborsClassifier Precision: 0.24964739069111425
GaussianNB Precision: 0.16881998277347116
SVM Precision: 0.15458937198067632
Random Forest Precision: 0.3944954128440367
Tuned Random Forest Precision: 0.410958904109589
AdaBoost Precision: 0.37398373983739835
Tuned AdaBoost Precision: 0.4669187145557656
GradientBoosting Precision: 0.4139344262295082
Tuned GradientBoosting Precision: 0.4846153846153846
XGB Precision: 0.4548736462093863
Tuned XGB Precision: 0.476303317535545
LGBM Precision: 0.5075187969924813
Tuned LGBM Precision: 0.5019157088122606
CatBoost Precision: 0.5059760956175299
Tuned CatBoost Precision: 0.5108695652173914


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_sm, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_sm, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_sm, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_sm, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_sm, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_sm, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_sm, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_sm, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_sm, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_sm, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_sm, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_sm, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_sm, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_sm, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_sm, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_sm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_sm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_sm, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_sm, zero_division= 0)
                  }

smote_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
smote_precision= smote_precision.sort_values(by= 'Precision Score', ascending=False)
print(smote_precision)

                                    Model  Precision Score
18              Tuned CatBoost Precision:         0.510870
15                        LGBM Precision:         0.507519
17                    CatBoost Precision:         0.505976
16                  Tuned LGBM Precision:         0.501916
12      Tuned GradientBoosting Precision:         0.484615
14                   Tuned XGB Precision:         0.476303
10              Tuned AdaBoost Precision:         0.466919
13                         XGB Precision:         0.454874
11            GradientBoosting Precision:         0.413934
8          Tuned Random Forest Precision:         0.410959
7                Random Forest Precision:         0.394495
9                     AdaBoost Precision:         0.373984
2          Tuned Decision Tree Precision:         0.294118
0          Logistic Regression Precision:         0.255652
4   Tuned KNeighborsClassifier Precision:         0.249647
1                Decision Tree Precision:         0.2435

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_sm),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_sm),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_sm),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_sm),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_sm),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_sm),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_sm),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_sm),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_sm),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_sm),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_sm),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_sm),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_sm),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_sm),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_sm),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_sm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_sm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_sm),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_sm)
                  }

smote_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
smote_accuracy= smote_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(smote_accuracy)

                                   Model  Accuracy Score
18              Tuned CatBoost Accuracy:        0.903913
15                        LGBM Accuracy:        0.903701
17                    CatBoost Accuracy:        0.903595
16                  Tuned LGBM Accuracy:        0.903383
12      Tuned GradientBoosting Accuracy:        0.902004
14                   Tuned XGB Accuracy:        0.901156
13                         XGB Accuracy:        0.900626
10              Tuned AdaBoost Accuracy:        0.899565
8          Tuned Random Forest Accuracy:        0.897762
7                Random Forest Accuracy:        0.895959
11            GradientBoosting Accuracy:        0.889914
6                          SVM Accuracy:        0.888111
9                     AdaBoost Accuracy:        0.873688
2          Tuned Decision Tree Accuracy:        0.873582
4   Tuned KNeighborsClassifier Accuracy:        0.865627
1                Decision Tree Accuracy:        0.846113
3         KNeighborsClassifier 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_sm),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_sm),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_sm),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_sm),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_sm),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_sm),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_sm),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_sm),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_sm),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_sm),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_sm),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_sm),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_sm),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_sm),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_sm),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_sm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_sm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_sm),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_sm)
                  }

smote_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
smote_auc= smote_auc.sort_values(by= 'AUC Score', ascending=False)
print(smote_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.851917
10              Tuned AdaBoost AUC:   0.848556
12      Tuned GradientBoosting AUC:   0.846054
16                  Tuned LGBM AUC:   0.845202
15                        LGBM AUC:   0.844257
14                   Tuned XGB AUC:   0.844233
11            GradientBoosting AUC:   0.843950
9                     AdaBoost AUC:   0.842642
17                    CatBoost AUC:   0.841640
18              Tuned CatBoost AUC:   0.838540
8          Tuned Random Forest AUC:   0.834386
13                         XGB AUC:   0.832605
7                Random Forest AUC:   0.823986
5                   GaussianNB AUC:   0.798275
3         KNeighborsClassifier AUC:   0.739022
2          Tuned Decision Tree AUC:   0.722412
6                          SVM AUC:   0.697322
4   Tuned KNeighborsClassifier AUC:   0.622678
1                Decision Tree AUC:   0.592835
